In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from datetime import timedelta as td
drive.mount('/content/drive')
import warnings

warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
!pip install pandas_flavor
!pip install xgboost==1.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 MB 7.5 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.7.4
    Uninstalling xgboost-1.7.4:
      Successfully uninstalled xgboost-1.7.4


### Defining Constants

In [ ]:
coops_top11 = [
'kansas city nebraska sioux city omaha',
'sacramento fresno reno bakersfield',
'washington dc baltimore eastern shore',
'dallas tyler',
'denver col springs s colorado',
'triad wilmington sw va wi s gnw',
'albuquerque el paso',
'atlanta nw georgia',
'pittsburgh johnstown erie',
'philadelphia',
'st louis',

]


all_coops=['denver col springs s colorado',
 'new england',
 'terrehaute evansville cape gir paducah',
 'st louis',
 'philadelphia',
 'tampa st pete orlando',
 'pittsburgh johnstown erie',
 'portland medford eugene bend',
 'memphis mississippi',
 'south la',
 'miami w palm beach ft myers',
 'kansas city nebraska sioux city omaha',
 'washington dc baltimore eastern shore',
 'salt lake city s wyoming',
 'chicago',
 'atlanta nw georgia',
 'triad wilmington sw va wi s gnw',
 'greater indiana',
 'albuquerque el paso',
 'dallas tyler',
 'sacramento fresno reno bakersfield',
 'san francisco santa barbara',
 'little rock shreveport monroe',
 'wichita topeka ok city wichita falls',
 'seattle tacoma',
 'sw north carolina e south carolina',
 'tulsa fort smith',
 'savannah augusta macon albany columbus',
 'san antonio austin waco victoria',
 'w virginia knoxville bristol kingsport',
 'wisconsin rockford',
 'richmond shenandoah valley norfolk',
 'lexington louisville',
 'houston beaumont',
 'valley corpus christi laredo',
 'susquehanna',
 'jacksonville tallahassee',
 'cleveland',
 'michigan',
 'des moines quad cities peoria cent il',
 'alabama nw florida',
 'alaska',
 'montana',
 'new york metro',
 'columbus cincy dayton toledo lima',
 'minneapolis st paul rochester duluth',
 'joplin quincy springfield col jeff city',
 'hawaii',
 'albany buffalo rochester syracuse',
 'greater tennessee valley',
 'n dakota s dakota w minnesota',
 'los angeles san diego',
 'e washington nw idaho ne oregon',
 'boise twin falls idaho falls',
 'phoenix las vegas tucson yuma el centro',
 'amarillo lubbock']
s3=False
LTO_ITEMS =[2930,1751,2931,
1270,
6244,
118,
557,
14962,
657,
14938,
14970,
2190,
5909,
3573,
1120,
1921,
2157,
2108,
428,
2133,
3783,
14960,
14924,
2936,
2156,
2100,
427,
9727,
2127,
3780,
14959,
14889,
14888,
10,
8971,
4257,
14925,
601,
5910,
1926,
6260,
2088,
424,
4055,
3779,
14958,
14926,
14929,
14869,
558,
126,
782,
1014,
1000,
1012,
457,
258,
462,
485,
1975,
1121,
6763
]
start_date = "2019-01-01"
end_date = "2023-01-27"
exclusion_dates = ['2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', 
                   '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', 
                   '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', 
                   '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', 
                   '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', 
                   '2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07', 
                   '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', 
                   '2020-04-12', '2020-04-13', '2020-04-14', '2020-04-15', 
                   '2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19', 
                   '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23', 
                   '2020-04-24', '2020-04-25', '2020-04-26', '2020-04-27', 
                   '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01', 
                   '2020-05-02', '2020-05-03', '2020-05-04', '2020-05-05', 
                   '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-09', 
                   '2020-05-10', '2020-05-11', '2020-05-12', '2020-05-13', 
                   '2020-05-14', '2020-05-15', '2020-05-16', '2020-05-17', 
                   '2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21', 
                   '2020-05-22', '2020-05-23', '2020-05-24', '2020-05-25', 
                   '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29', 
                   '2020-05-30', '2020-05-31', '2020-06-01', '2020-06-02', 
                   '2020-06-03', '2020-06-04', '2020-06-05', '2020-06-06', 
                   '2020-06-07', '2020-06-08', '2020-06-09', '2020-06-10', 
                   '2020-06-11', '2020-06-12', '2020-06-13', '2020-06-14', 
                   '2020-06-15', '2020-06-16', '2020-06-17', '2020-06-18', 
                   '2020-06-19', '2020-06-20', '2020-06-21', '2020-06-22', 
                   '2020-06-23', '2020-06-24', '2020-06-25', '2020-06-26', 
                   '2020-06-27', '2020-06-28', '2020-06-29', '2020-06-30',
                      '2022-07-04','2022-07-11','2022-07-18','2022-07-25']

DELISTED_ITEMS = [4775, 14932, 14885]

## Data Prep

In [ ]:
import json
import os
import pickle
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from tqdm import tqdm
from xgboost import XGBRegressor
#from ta_lib.usa.modelling import gc_model as gcm
import shutil
#from ta_lib.usa import model_constants


import time 
import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas_flavor as pf


@pf.register_dataframe_method
def pd_cols_to_datetime(df, col_='pos_busn_dt',dt_format=None):
    '''Convert a column in pandas dataframe to datetime.

    Parameters
    ----------
    df: pd.Dataframe
    col_ : str, default is "pos_busn_dt"
    dt_format : str, default is None
        Same as the format parameter in pd.to_datetimeb 

    Returns
    -------
    df : pd.Dataframe
    '''

    if dt_format is None:
        df[col_] = pd.to_datetime(df[col_])
    else:
        df[col_] = pd.to_datetime(df[col_],format=dt_format)
    return df
@pf.register_dataframe_method
def get_week_start_dt(df, date_col_name='pos_busn_dt'):
    '''Get week start date for date columns columns in pandas dataframe to datetime.

    Parameters
    ----------
    df: pd.Dataframe
    date_col_name : str, default is "pos_busn_dt"

    Returns
    -------
    df : adds `week_st_dt` column to dataframe
    '''

    df['week_st_dt'] = df[date_col_name].apply(lambda x: x - td(days=x.weekday()))
    return df

### Config

In [ ]:
cfg_refit_model =False
generate_training_dataset=False
save_gc_predictions = False
save_gc_elasticitity=False

#### Scaling for category

In [ ]:

def scaling_for_category(df_demand_predictions,ITEM_MASTER,pred_col):

  
  df_item_master = pd.read_csv(ITEM_MASTER).rename(columns={'sld_menu_itm_id':'dv_id'})
  df_demand_predictions  = df_demand_predictions.merge(df_item_master[['item_category','dv_id']],on=['dv_id'],how='left')
  str_count_path = os.path.join(ONE_DATA_ROOT,'y_data','04_base_price_asp','raw','coop=')
  print(str_count_path)
  df_train=pd.DataFrame()

  for coop in tqdm(coops_top11):

    #for train 
    str_cnt_df = pd.read_parquet(str_count_path+coop).rename(columns={'sld_menu_itm_id':'dv_id','week_st_dt':'pos_busn_dt'}).assign(coop_name=coop)[['dv_id','pos_busn_dt','store_count','coop_name']]
    str_cnt_df = str_cnt_df.merge(df_item_master[['item_category','dv_id']],on=['dv_id'],how='left')
    str_cnt_df['pos_busn_dt']=pd.to_datetime(str_cnt_df['pos_busn_dt'])
    
    str_cnt_grouped = str_cnt_df.groupby(['coop_name','pos_busn_dt','item_category'])['store_count'].max().reset_index()
    str_cnt_grouped['pos_busn_dt']=pd.to_datetime(str_cnt_grouped['pos_busn_dt'])
    
    df_demand_coop_train = df_demand_predictions.query('coop_name==@coop and split=="train"')
    df_demand_coop_train =df_demand_coop_train.merge(str_cnt_df[['coop_name','pos_busn_dt','dv_id','store_count']],on=['coop_name','pos_busn_dt','dv_id'],how='left')
    df_demand_coop_train[pred_col] = df_demand_coop_train[pred_col]*df_demand_coop_train['store_count']
    df_demand_coop_train.drop(columns=['store_count'],inplace=True)

    df_demand_coop_train= df_demand_coop_train.merge(str_cnt_grouped,on=['coop_name','pos_busn_dt','item_category'],how='left')
    df_demand_coop_train[pred_col] =df_demand_coop_train[pred_col]/df_demand_coop_train['store_count']

    df_demand_coop_train.drop(columns=['store_count','item_category'],inplace=True)


    #for test 
    str_cnt_df_dv_id=str_cnt_df.copy()
    str_cnt_df_category=str_cnt_df.copy()
    

    df_demand_coop_test = df_demand_predictions.query('coop_name==@coop and split=="test"')
    str_cnt_df_dv_id['rank']=str_cnt_df_dv_id.groupby(["coop_name","dv_id"])["pos_busn_dt"].rank(method="first", ascending=False)

    str_cnt_df_dv_id =str_cnt_df_dv_id.query('rank==1.0')

    df_demand_coop_test =df_demand_coop_test.merge(str_cnt_df_dv_id[['coop_name','dv_id','store_count']],on=['coop_name','dv_id'],how='left')
    df_demand_coop_test[pred_col] = df_demand_coop_test[pred_col]*df_demand_coop_test['store_count']
    df_demand_coop_test.drop(columns=['store_count'],inplace=True)
    
    str_cnt_df_category['rank'] = str_cnt_df_category.groupby(["coop_name","item_category"])["pos_busn_dt"].rank(method="first", ascending=False)
    str_cnt_df_category =str_cnt_df_category.query('rank==1.0')
    str_cnt_df_category = str_cnt_df_category[['coop_name','item_category','store_count']]

    df_demand_coop_test= df_demand_coop_test.merge(str_cnt_df_category,on=['coop_name','item_category'],how='left')
    df_demand_coop_test[pred_col] =df_demand_coop_test[pred_col]/df_demand_coop_test['store_count']
    df_demand_coop_test.drop(columns=['store_count','item_category'],inplace=True)

    df_demand_coop_train=df_demand_coop_train.append(df_demand_coop_test,ignore_index=True).reset_index(drop=True)

    df_train=df_train.append(df_demand_coop_train,ignore_index=True)

  return df_train.reset_index(drop=True)

In [ ]:
def return_top_items(item_master_org_path,offer_path,item_mapping,item_master_scaling_path):

  item_master_org=pd.read_excel(item_master_org_path,sheet_name='Item_Master').query('LTO_Flag==0')[['instore transaction','sld_menu_itm_id','instore units','instore net sales']]
  
  item_master_org['sld_menu_itm_id']=item_master_org['sld_menu_itm_id'].astype(np.int64).astype(str)
  ## sp sp mapping 
  df_item_grp = pd.read_csv(item_mapping)
  df_item_grp['sld_menu_itm_id']=df_item_grp['sld_menu_itm_id'].astype(np.int64).astype(str)
  item_master_org = item_master_org.merge(df_item_grp,on = ['sld_menu_itm_id'],how = 'left')
  fil_ = item_master_org.group_id.isnull()
  item_master_org.loc[fil_, "group_id"] = item_master_org.loc[fil_, "sld_menu_itm_id"]
  item_master_org['group_id']=item_master_org['group_id'].astype(np.int64).astype(str)

  item_master_org.drop(columns=['sld_menu_itm_id'],inplace=True)
  item_master_org.rename(columns={'group_id':'sld_menu_itm_id'},inplace=True)

  ## item master scaling maping 

  item_master_scaling=pd.read_parquet(item_master_scaling_path).dropna(subset=['Base_item_ID'])
  item_master_scaling['Base_item_ID']=item_master_scaling['Base_item_ID'].astype(np.int64).astype(str)
  item_master_scaling['sld_menu_itm_id']=item_master_scaling['sld_menu_itm_id'].astype(np.int64).astype(str)
  item_master_org=item_master_org.merge(item_master_scaling,on=['sld_menu_itm_id'],how='left')
  item_master_org['sld_menu_itm_id']= np.where(item_master_org['Base_item_ID'].isna(),item_master_org['sld_menu_itm_id'],item_master_org['Base_item_ID'])
  item_master_org.drop(columns=['Scaling_Factor','Base_item_ID'],inplace=True)
 

  def return_top_100(df,col):
      df = df.dropna(subset=[col,'sld_menu_itm_id'])
      df['rank']=df[col].rank(method="first", ascending=False)
      df = df.sort_values('rank')
      return list(set(df.head(100)['sld_menu_itm_id']))
  final_items=[]
  
  selection_cols = ['instore transaction','instore units','instore net sales']
  for sol in selection_cols:   
      final_items.extend(return_top_100(item_master_org,sol))
      print(final_items)
  
  final_items=list(map(int,final_items))
 
  offer_item_freq= dict()
  root_dir = offer_path
  for fil in os.listdir(root_dir):
      offers=[col.split('_')[2] for col in pd.read_parquet(os.path.join(root_dir,fil)).columns if col.startswith('dd_')]
      for offer in offers:
          if offer!='off':
              temp_items = list(map(int,offer.replace('|','-').split('-')))
              for item in temp_items:
                  if item not in offer_item_freq:
                    if item not in LTO_ITEMS:
                      offer_item_freq[item]=1
                  else:
                    if item not in LTO_ITEMS:
                      offer_item_freq[item]=offer_item_freq[item]+1
                      
   
  top_100_items  = set(list(dict(sorted(offer_item_freq.items(), key=lambda item: item[1],reverse=True)[:100]).keys()))
  item_select = set(final_items).union(top_100_items)
  item_select=list(map(int,item_select))
  return item_select
      

### utility function

In [ ]:

def _get_feature_cat_mapper(item_master_path):
    item_master = pd.read_csv(item_master_path)
    item_master["feature_name"] = item_master["sld_menu_itm_id"].apply(
        lambda x: "item_" + str(x)
    )
    feature_mapper = item_master.set_index("sld_menu_itm_id")["feature_name"].to_dict()
    cat_mapper = item_master.set_index("feature_name")["item_category"].to_dict()
    return feature_mapper, cat_mapper



def items_to_model():
    item_exclusions = LTO_ITEMS + DELISTED_ITEMS
    df_item = pd.read_csv(ITEM_MASTER)
    df_item_grp = pd.read_csv(ITEM_MAPPING)
    df_item_group = df_item.merge(df_item_grp,on = ['sld_menu_itm_id'],how = 'left')
    df_item_group['group_id'].fillna(df_item_group['sld_menu_itm_id'],inplace = True)
    list_items = list(df_item_group['group_id'].unique().astype(int))
    list_items = list (set(list_items)- set(item_exclusions))
    return list_items
    
def wmape(y_true, y_pred):
    """Computes Weighted Mean Absolute Deviation.

    Similar to mape but weighted by y_true.
    This ignores y_true=0 and tends to be higher
    if the errors are higher for higher y_true values.

    Parameters
    ----------
    y_true: actual values
    y_pred: predicted values
    """
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)


wmape_scorer = make_scorer(wmape, greater_is_better=False)

def load_item_qty(y_data_path, item_master_path, item_grp_path, 
                  bundle_config_path, item_master_scaling_pth
                  ,coop=None,start_date="2018-01-01", 
                  end_date="2022-07-31", item=None,store_id=None,store_grp="False",
                  store_lvl="False",store_coop="False", bundle_config_flag=True,
                  varient_mapping=True,scaling=True):
     
    '''Load quantity data by filtering specific dates and segments.

    Parameters
    ----------
    item_master : pd.DataFrame
        DataFrame that contains items to be considered
    item_grp : pd.DataFrame
        DataFrame that contains parent grouping to the item_ids
    start_date : str, default is "2021-01-01".
        start date to filter data 
    end_date : str, default is "2022-03-31"
        end_date to filter data.
    path : str,
        pass path of the loaction where data is kept.
    coop : str,
        pass coop name, list of coops for which data to be loaded.
        

    Returns
    -------
    Grouped data frame with specified filters.
    '''
    
    # store_grp = model_constants.STORE_GROUPING
    item_master = pd.read_csv(item_master_path)
    item_grp = pd.read_csv(item_grp_path)

    
    if store_grp=="True" or store_lvl=='True' or store_coop=="True":
        cols_ = ['mcd_gbal_lcat_id_nu','pos_busn_dt','sld_menu_itm_id', 'quantity', 'net_sales']
    else:    
        cols_ = ['pos_busn_dt', 'sld_menu_itm_id', 'quantity']
    fils_ = []

    if type(coop) is list:
        fils_.append(('coop', 'in', set(coop))) 
    elif type(coop) is str:
        if coop.lower() == "national" :
            pass #No filter on coops and load all
        else:
            fils_.append(('coop', '=', coop))
    else:
        raise ValueError('Coop name is not correct')
    
    if start_date is not None:
        fils_.append(('pos_busn_dt', '>=', start_date))
    if end_date is not None:
        fils_.append(('pos_busn_dt', '<=', end_date))
    # add item filters

    # if item is not None:
    #     items = item_grp[item_grp.group_id.isin(item)].sld_menu_itm_id.to_list()
    #     fils_.append(('sld_menu_itm_id','in',set(items)))

    # if store_id is not None:
    #     cols_.append('mcd_gbal_lcat_id_nu')
    if varient_mapping == True:
        if s3:
            prefix_objs = bucket.objects.filter(Prefix="Pricing/cleaned/Quantity/coop_item_week")
            for obj in prefix_objs:
                if 'SUCCESS' not in obj.key:
                    path1 = f's3://{obj.bucket_name}/{obj.key}'
                    #df = pd.read_parquet(path1)
                    geo_name=obj.key.split("=")[1].split("/")[0].strip().upper()
                    #print(geo_name)
                    #print(geo_name)
                    if geo_name=='unknown'.upper():
                        continue
                    if geo_name==coop.upper():
                        print(geo_name,coop)
                        break
                    else:
                        continue
        # df = (
        #     pq.ParquetDataset(path1, filters=fils_)
        #     .read_pandas(columns=cols_)
        #     .to_pandas()
        #     .merge(item_grp[["sld_menu_itm_id", "group_id"]],on="sld_menu_itm_id",how="left")
        # )
            df = pd.read_parquet(path1,columns=cols_,filters=fils_)
            df = pd.merge(df,item_grp[["sld_menu_itm_id", "group_id"]],on="sld_menu_itm_id",how="left")
            df['coop'] = coop
            #print(df.head(10))
            print("yes")
            fil_ = df.group_id.isnull()
            df.loc[fil_, "group_id"] = df.loc[fil_, "sld_menu_itm_id"]

        else:
           df = (
            pq.ParquetDataset(y_data_path, filters=fils_)
            .read_pandas(columns=cols_)
            .to_pandas()
            .merge(item_grp[["sld_menu_itm_id", "group_id"]],on="sld_menu_itm_id",how="left")
        )
        df['coop'] = coop
        fil_ = df.group_id.isnull()
        df.loc[fil_, "group_id"] = df.loc[fil_, "sld_menu_itm_id"]
   

    else:
        df = (
            pq.ParquetDataset(y_data_path, filters=fils_)
            .read_pandas(columns=cols_)
            .to_pandas()
        )
        fil_ = df.group_id.isnull()
        df.loc[fil_, "group_id"] = df.loc[fil_, "sld_menu_itm_id"]

    if store_lvl=='True':
        df=df[df['mcd_gbal_lcat_id_nu']==store_id].reset_index(drop = True)
        
    df['pos_busn_dt'] = df['pos_busn_dt'].astype(str)
    df = df[(df['pos_busn_dt']>=start_date) & (df['pos_busn_dt']<=end_date)].reset_index(drop = True) 

   
    if scaling and item_master_scaling_pth is not None :
     
        #item_master_scaling = pd.read_parquet(item_master_scaling_pth).query("Base_item_ID!='nan'")
        item_master_scaling = pd.read_parquet(item_master_scaling_pth).dropna(subset=['Base_item_ID'])#.query("Base_item_ID!='nan'")
        item_master_scaling['Base_item_ID']=item_master_scaling['Base_item_ID'].astype(np.int64).astype(str)
        base_id_sf = item_master_scaling[['Scaling_Factor','sld_menu_itm_id']].set_index('sld_menu_itm_id').to_dict()['Scaling_Factor']
        base_id_child = item_master_scaling[['sld_menu_itm_id','Base_item_ID']].set_index('sld_menu_itm_id').to_dict()['Base_item_ID']

        df_new=df.copy()

        for item_ in base_id_child.keys():

            parent =  int(base_id_child[item_])

            if df_new.query("sld_menu_itm_id==@item_").shape[0]>0 and df_new.query("sld_menu_itm_id==@parent").shape[0]>0:

                
                df_new['quantity'] = np.where(df_new['sld_menu_itm_id']==item_,base_id_sf[item_]*df_new['quantity'], df_new['quantity'])

        

                group_id_parent = df_new[df_new['sld_menu_itm_id']==parent]['group_id'].values[0]

                df_new['group_id'] = np.where(df_new['sld_menu_itm_id']==item_,group_id_parent,df_new['group_id'])

                df_new['sld_menu_itm_id'] = np.where(df_new['sld_menu_itm_id']==item_,parent, df_new['sld_menu_itm_id'].astype(np.int))

               

        
        df=df_new



    #df = pd.merge(df,store_binning,on='mcd_gbal_lcat_id_nu',how='left')
    #df.drop(columns=['mcd_gbal_lcat_id_nu'],inplace=True)
    #df.rename(columns={'median_minmax_bins':'mcd_gbal_lcat_id_nu'},inplace=True)
    # Grouping
    
    df = pd_cols_to_datetime(df)
    
    if bundle_config_flag:
        df['group_id'] = df['group_id'].astype(int)
        #df['sld_menu_itm_id'] = df['sld_menu_itm_id'].astype(int)
        df['pos_busn_dt']=pd.to_datetime(df['pos_busn_dt'])
        df_bundle_mapping = pd.read_csv(bundle_config_path,usecols=['coop','pos_busn_dt','parent_id','child_id','qty_weight','sales_weight'])
        df_bundle_mapping['coop']=df_bundle_mapping['coop'].apply(lambda x :str(x).lower().strip())
        df_bundle_mapping['pos_busn_dt']=pd.to_datetime(df_bundle_mapping['pos_busn_dt'])
        df_bundle_mapping['parent_id'] = df_bundle_mapping['parent_id'].astype(int)
        
        df=df.merge(df_bundle_mapping,left_on=['coop','pos_busn_dt','group_id'],right_on = ['coop','pos_busn_dt','parent_id'],how='left')
        
        #df['sld_menu_itm_id']=np.where(df['child_id'].isna(),df['sld_menu_itm_id'],df['child_id'])
        df['group_id']=np.where(df['child_id'].isna(),df['group_id'],df['child_id'])        
        
        df['quantity']=np.where(df['child_id'].isna(),df['quantity'],df['quantity']*df['qty_weight'])
        # df['offer_qty']=np.where(df['child_id'].isna(),df['offer_qty'],df['offer_qty']*df['qty_weight'])
       
        df.drop(columns=['child_id','qty_weight','sales_weight','parent_id'],inplace=True)
    
    df['group_id']=df['group_id'].astype(int)
    df.coop = df.coop.astype(str)
    
    #filtering top items if not item list defined
    if item is None:
        li_items = item_master.sld_menu_itm_id.to_list()
        df = df.query("group_id in @li_items")
    else:
        df = df.query("group_id in @item")

    if type(coop) is str:
        if store_grp=='True' or store_lvl=='True' or store_coop=='True':
            df = (
            df.groupby(["mcd_gbal_lcat_id_nu","pos_busn_dt", "group_id"])
            .agg(
                quantity=('quantity','sum'),
                #net_sales=('net_sales','sum'),
            )
            .reset_index()
            .rename(columns={"group_id": "dv_id"})
            .get_week_start_dt()
        )
            if coop.lower() == 'national':
                df['coop'] = "National"
            else:
                df['coop'] = coop
        else:
            df = (
                df.groupby(["pos_busn_dt", "group_id"])
                .agg(
                    quantity=('quantity','sum'),
                    #net_sales=('net_sales','sum'),
                )
                .reset_index()
                .rename(columns={"group_id": "dv_id"})
                .get_week_start_dt()
            )
            if coop.lower() == 'national':
                df['coop'] = "National"
            else:
                df['coop'] = coop
    else :
         df = (
            df.groupby(["coop","pos_busn_dt", "group_id"])
            .agg(
                quantity=('quantity','sum'),
                #net_sales=('net_sales','sum'),
            )
            .reset_index()
            .rename(columns={"group_id": "dv_id"})
            .get_week_start_dt()
        )
    
    return df

def load_item_qty_data(df_item_qty_path,item_master_path,item_grp_path,coop,
                       start_date,end_date,item,exclusion_dates = [],
                       store_id=None,store_coop=None,bundle_config_path = None,
                       scaling=True,item_master_scaling_path = None):

    """reads the item quantity path for a particular coop and returns the df with some cosmetic changes"""


    store_grp="False"
    store_lvl="False"
    store_coop="False"


    df = load_item_qty(
        y_data_path=df_item_qty_path,
        item_master_path=item_master_path,
        item_grp_path=item_grp_path,
        coop=coop,
        start_date = start_date,
        end_date = end_date,
        item=item,
        store_id=store_id,
        store_grp = store_grp,
        store_lvl = store_lvl,
        store_coop = store_coop,
        bundle_config_path=bundle_config_path,
        scaling= scaling,
        item_master_scaling_pth = item_master_scaling_path,
    )
    df["pos_busn_dt"] = pd.to_datetime(df["pos_busn_dt"])
    if len(exclusion_dates) > 0:
        df = df[~df.pos_busn_dt.isin(exclusion_dates)]
    rename_dict = {"sld_menu_itm_id": "dv_id", "quantity": "y"}
    df.rename(columns=rename_dict, inplace=True)
    if store_grp=='True' or store_lvl=='True' or store_coop == "True":
        df = df[["mcd_gbal_lcat_id_nu","pos_busn_dt", "dv_id", "y"]]
        df["estimated_measure"] = "item_qty"
    else:
        df = df[["pos_busn_dt", "dv_id", "y"]]
        df["estimated_measure"] = "item_qty"

    return df

def load_gc_data(coop_up,start_date, end_date, gc_path_weekly,
                exclusion_dates = [],s3=False,granularity='weekly',
                 gc_path_daily=None):
    
    """reads the gc path for a particular coop and returns the df with some cosmetic changes"""
    
    if s3:
        import boto3
        s3 = boto3.resource('s3')
        bucket = s3.Bucket('us-east-1-cprod-us-spark-cluster')
        prefix_objs = bucket.objects.filter(Prefix="Pricing/cleaned/GC/coop_day")
        for obj in prefix_objs:
            if 'SUCCESS' not in obj.key:
                path1 = f's3://{obj.bucket_name}/{obj.key}'
                df = pd.read_parquet(path1)
                geo_name=obj.key.split("=")[1].split("/")[0].strip().upper()
                #print(geo_name)
                if geo_name=='unknown'.upper():
                    continue
                if geo_name==coop_up:
                    print(geo_name,coop_up)
                    break
                else:
                    continue
      
        df['coop']=coop_up.lower()
        df['pos_busn_dt']=pd.to_datetime(df['pos_busn_dt'])
        df =df.query("pos_busn_dt>=@start_date and pos_busn_dt<=@end_date").reset_index(drop=True)

        

        if len(exclusion_dates) > 0:
            df = df[~df.pos_busn_dt.isin(exclusion_dates)]
        rename_dict = { "total_gc": "y"}
        df.rename(columns=rename_dict, inplace=True)
        df = df.sort_values("pos_busn_dt")
        df = df[["pos_busn_dt", "coop", "y"]]
        df["estimated_measure"] = "gc"
        df=df.reset_index(drop=True)
        #display(df.head())

    else:

        if granularity=='daily':

            print(gc_path_daily,coop_up.lower()+'.parquet')
            df = pd.read_parquet(gc_path_daily+coop_up.lower())
            

        else:
            print(gc_path_weekly+coop_up.lower()+'.parquet')
            df = pd.read_parquet(gc_path_weekly+coop_up.lower())
            
        #print(df)
        df['coop_name']=coop_up.lower()
        df['pos_busn_dt']=pd.to_datetime(df['pos_busn_dt'])

        df=df.query("pos_busn_dt>=@start_date and pos_busn_dt<=@end_date")
        if len(exclusion_dates) > 0:
            df = df[~df.pos_busn_dt.isin(exclusion_dates)]
        else:
            pass

        rename_dict = { "total_gc": "y"}
        df.rename(columns=rename_dict, inplace=True)
        df = df.sort_values("pos_busn_dt")
        df = df[["pos_busn_dt", "coop_name", "y"]]
        df["estimated_measure"] = "gc"
        df=df.reset_index(drop=True)
                       
    return df

#### Generating Dataset

In [ ]:
def generate_dataset(data_iter,
                     output_path,
                     qty_data_path,
                     gc_data_path,
                     item_grp_path,
                     bundle_config_path,
                     item_master_scaling,
                     item_master_path,
                     coops_list,
                     new_exclusion_dates=None,
                     save_train_data=True,
                      ):
  
    if new_exclusion_dates is not None:
      exclusion_dates.extend(new_exclusion_dates)

    df_item_df = pd.DataFrame()
    df_category_df = pd.DataFrame()

    df_train = pd.DataFrame()


    list_items = items_to_model()

    topitems = return_top_items(ITEM_MASTER_ORG,X_OFFER_PATH,ITEM_MAPPING,ITEM_MASTER_SCALING)

    topitems= [i for i in topitems if i in list_items]

    print(len(list_items))
    print(list_items)

    print((topitems))
    print(len(topitems))
    
    #category_items =  list(set(list_items)-set(topitems))


    df1 = pd.DataFrame({'all_items':list_items})
    df2 = pd.DataFrame({'top_items':list(topitems)})


    items_log = pd.concat([df1,df2],axis=1)

 

    item_master_df= pd.read_csv(item_master_path)
    item_master_df.rename(columns={'sld_menu_itm_id':'dv_id'},inplace=True)


    for coop in tqdm(coops_list):


        print('---------------------------------')
        print(f"Total COOPs : {len(coops_list)}")
        print(f"Current COOP Name : {coop}")

        ## load y data.
        #top_items = list(item_config_analysis.query('Coop==@coop')['sld_menu_itm_id'])


        st = time.time()    
        y_qty = load_item_qty_data(df_item_qty_path = qty_data_path,
                                   item_master_path = item_master_path,
                                   item_grp_path = item_grp_path,
                                   coop = coop,
                                   start_date = start_date,
                                   end_date = end_date,item = list_items,
                                   exclusion_dates = exclusion_dates,
                                   bundle_config_path=bundle_config_path,
                                   item_master_scaling_path=item_master_scaling)\
                                  .rename(columns={'y':'actual_qty'})\
                                  .assign(estimated_measure='gc')\
                                  .assign(coop_name=coop)

        y_qty['dv_id'] = y_qty['dv_id'].astype(np.int64)

        y_gc= load_gc_data(coop.upper(), start_date, end_date,gc_data_path,
                                exclusion_dates,s3=False,granularity="weekly")
        
        base_price_cost = os.path.join(ONE_DATA_ROOT,'y_data','04_base_price_asp','raw',f'coop={coop}')

        base_price_cost_denver = pd.read_parquet(base_price_cost).drop_duplicates() 
        base_price_cost_denver.rename(columns={'sld_menu_itm_id':'dv_id','week_st_dt':'pos_busn_dt'},inplace=True)
        base_price_cost_denver['pos_busn_dt']=pd.to_datetime(base_price_cost_denver['pos_busn_dt'])
        base_price_cost_denver['dv_id']=base_price_cost_denver['dv_id'].astype(int)

        str_count_max=base_price_cost_denver.groupby(['pos_busn_dt'])['store_count'].max().reset_index()


    
        y_qty = y_qty.merge(base_price_cost_denver[['dv_id','store_count','pos_busn_dt']],on=['pos_busn_dt','dv_id'],how='left')
        y_qty['actual_qty'] = y_qty['actual_qty']/y_qty['store_count']

        y_qty.drop(columns=['store_count'],inplace=True)

        y_gc=y_gc.merge(str_count_max,on=['pos_busn_dt'],how='left')

        y_gc['y'] = y_gc['y']/y_gc['store_count']

        y_gc.drop(columns=['store_count'],inplace=True)




        ############ data prep ##########################
        ################ category ########################

        df_x_category = y_qty.merge(item_master_df[['dv_id','item_category']],on=['dv_id'],how='left').dropna()
        df_x_category['estimated_measure'] ='gc'
        
        df_x_category = pd.pivot_table(df_x_category,values='actual_qty',columns='item_category',index=['pos_busn_dt','coop_name','estimated_measure'],aggfunc='sum').reset_index()
        
        ############# item ##########################
        y_qty=y_qty.query('dv_id in @topitems')

        y_qty['dv_id']='item_'+y_qty['dv_id'].astype(str)
        df_x_item = pd.pivot_table(y_qty,values='actual_qty',columns='dv_id',index=['pos_busn_dt','coop_name','estimated_measure'],aggfunc='sum').reset_index()

        ################# addding y data in x matrix ###################### 
        df1 = df_x_category.merge(y_gc,on=['coop_name','pos_busn_dt','estimated_measure'],how='inner')

        df2 = df_x_item.merge(y_gc,on=['coop_name','pos_busn_dt','estimated_measure'],how='inner')

        df_category_df= df_category_df.append(df1,ignore_index=True)

        df_item_df= df_item_df.append(df2,ignore_index=True)

        # df_x_y = df_x_category.merge(df_x_item,on=['coop_name','pos_busn_dt','estimated_measure'],how='outer')\
        #           .merge(y_gc,on=['coop_name','pos_busn_dt','estimated_measure'],how='outer')

        # df_train=df_train.append(df_x_y,ignore_index=True)
    
    cols_to_drop =[col for col in df_category_df.columns if 'MULTI' in col or 'multi' in col  ]

    df_category_df=df_category_df.drop(columns=cols_to_drop,axis=1)

    output_dir = os.path.join(output_path,data_iter)

    cat_dir = os.path.join(output_dir,'categories')
    item_dir = os.path.join(output_dir,'items')


    if os.path.exists(output_dir):
        pass
    else:
        os.makedirs(output_dir)
       

    if os.path.exists(cat_dir):
        pass
    else:
        os.makedirs(cat_dir)

    if os.path.exists(item_dir):
        pass
    else:
        os.makedirs(item_dir)

    df_item_df.to_parquet(os.path.join(item_dir,'train.parquet'),engine='pyarrow')
    df_category_df.to_parquet(os.path.join(cat_dir,'train.parquet'),engine='pyarrow')
    items_log.to_csv(os.path.join(output_dir,'item_logger.csv'),index=False)

    print("Dataset Created and saved @ ",output_dir)

    return df_item_df,df_category_df


In [ ]:

if __name__ == "__main__":

    #### CONFIG #####

    data_iter='14'
    coop_list = coops_top11
    
    data_prep_only=False
    
    cfg_demand_pipeline_iteration_num = 41
    
    cfg_gc_iteration_num = 18
    
    cfg_xgb_num_tasks = 1
    cfg_xgb_num_threads_per_task = 8
    

    if sys.platform == "win32":
        DRIVE_ROOT = r'G:\Shared drives'
    elif 'google.colab' in sys.modules:
        DRIVE_ROOT = '/content/drive/Shareddrives'
    else:
        DRIVE_ROOT = "/mnt/g/Shared drives"

    # ##### FOLDERS #####
    USDE_ROOT = os.path.join(DRIVE_ROOT, "MCD_Discount Engine", "US", "2022")
    
    

    MODELLING_ROOT = os.path.join(USDE_ROOT, "4. Modeling")
    ONE_DATA_ROOT = os.path.join(USDE_ROOT, "4. Modeling","01_data")

    OUTPUT_DATA_ROOT = os.path.join(MODELLING_ROOT, "01_data","y_data","02_gc_day_level","Coop X Item X Categories")
    XGB_GC_ROOT = os.path.join(MODELLING_ROOT,"03_gc_models","xgb_models")
    
    PIPELINE_ROOT = os.path.join(MODELLING_ROOT, "99_pipeline", str(cfg_demand_pipeline_iteration_num).zfill(2))
    DEMAND_ROOT = os.path.join(PIPELINE_ROOT, "01_demand")
    
    X_OFFER_PATH = os.path.join(ONE_DATA_ROOT,'x_offers','05_offer_adjusted','weekly','depth')

    ITEM_MASTER = os.path.join(ONE_DATA_ROOT,'item_master.csv')
    ITEM_MASTER_ORG  =  os.path.join(DRIVE_ROOT, "MCD_Discount Engine", "US", "2022","2. Delivery Tracks","EDA","item","item_master_internal_latest_v5.xlsx")
    ITEM_MAPPING = os.path.join(ONE_DATA_ROOT,'item_mapping.csv')
    BUNDLE_CONFIG_DATA = os.path.join(ONE_DATA_ROOT,'bundle_config_ydata.csv')
    ITEM_MASTER_SCALING= os.path.join(ONE_DATA_ROOT,"item_master_scaling.parquet")
    QTY_DATA_PATH = os.path.join(USDE_ROOT, "4. Modeling","01_data","y_data",
                                  "01_item_day_level","coop","weekly")
    GC_DATA_PATH_DAILY = os.path.join(USDE_ROOT, "4. Modeling","01_data","y_data",
                                      "02_gc_day_level","Coop X item X day","coop=")

    GC_DATA_PATH_WEEKLY = os.path.join(USDE_ROOT, "4. Modeling","01_data",
                                       "y_data","02_gc_day_level",
                                       "Coop X item X week","coop=")
    
    # #generate_dataset(data_iter,coop_list)

    ITEM_INFO = os.path.join(ONE_DATA_ROOT,'sp_opt','inputs','item_info_current.parquet')
    
    
    
    MODELLING_INPUTS = os.path.join(MODELLING_ROOT, "03_gc_models","xgb_models","parameters")
    
    #MODEL_ITERATION_ROOT = os.path.join(MODELLING_ROOT, "03_gc_models","xgb_models", str(cfg_gc_iteration_num))

    MODEL_ITERATION_ROOT = os.path.join(MODELLING_ROOT, "03_gc_models","xgb_models", str(cfg_gc_iteration_num))

    # MODEL_ITERATION_ROOT = os.path.join(
    #     USDE_ROOT, "5. Workspaces", "Bharath", "GC", "iterations", "17"
    # )

   
    output_gc_predictions_filepath = os.path.join(
        PIPELINE_ROOT, "06_gc_xgb", "model_predictions.csv"
    )
    output_gc_simulation_filepath = os.path.join(
        PIPELINE_ROOT, "06_gc_xgb", "gc_simulation_method_1.csv"
    )

    GC_DATA_ROOT = os.path.join(OUTPUT_DATA_ROOT,data_iter)

    # if os.path.exists(MODEL_ITERATION_ROOT):
    #     pass
    # else:
    #   time.sleep(5)
    #   shutil.copytree(MODELLING_INPUTS, MODEL_ITERATION_ROOT)
    

    # # ##### FILES #####

    # #fp_item_master = os.path.join(MODELLING_INPUTS, "item_master.csv")

    fp_gc_item_features = os.path.join(GC_DATA_ROOT,"items")
    fp_gc_category_features = os.path.join(GC_DATA_ROOT,"categories")

    fp_grid = os.path.join(MODEL_ITERATION_ROOT, "grid.csv")
    fp_demand_predictions = os.path.join(DEMAND_ROOT, 'model_collated', 'model_predictions.parquet')
    #output_tensor_path =  os.path.join(PIPELINE_ROOT,"04_tensor","naive_imputation","all_tensor_raw.csv")
    output_tensor_path =  os.path.join(PIPELINE_ROOT,"04_tensor","tensor_processed","all_tensor_processed_raw.csv")
    
    cl_coop_col = "coop_name"
    cl_response_col = "y"
    cl_date_col = "pos_busn_dt"
    cl_index_cols = [cl_date_col, "estimated_measure"]
    dict_feature_mapper, dict_cat_mapper = _get_feature_cat_mapper(item_master_path=ITEM_MASTER)

    #get store count as per info file

    #str_cnt_mapper = pd.read_parquet(ITEM_INFO).groupby(['coop_name'])['stores_count'].max().to_dict()

    
    


### Creating the train dataset 

In [ ]:
if generate_training_dataset:
  
  new_exclusions_dates= [str(i.date()) for i in pd.date_range('2022-07-01','2022-07-31')]

  generate_dataset( data_iter=data_iter,
                      output_path=OUTPUT_DATA_ROOT,
                      qty_data_path=QTY_DATA_PATH,
                   
                      gc_data_path =GC_DATA_PATH_WEEKLY,
                      item_grp_path=ITEM_MAPPING,
                      bundle_config_path=BUNDLE_CONFIG_DATA,
                      item_master_scaling=ITEM_MASTER_SCALING,
                      item_master_path=ITEM_MASTER,
                      coops_list=coops_top11,
                      new_exclusion_dates=new_exclusions_dates,
                      save_train_data=generate_training_dataset)
else:
    print("Training Dataset not created")

['1500', '6323', '8941', '8946', '9842', '706', '9839', '5926', '382', '78', '366', '991', '2844', '922', '3655', '973', '6975', '592', '276', '7002', '1', '8944', '520', '9071', '35', '5280', '8476', '334', '9840', '148', '252', '4314', '116', '8932', '1557', '46', '981', '279', '383', '31', '483', '4', '9088', '2842', '3430', '3', '61', '900', '85', '250', '3832', '6183', '473', '903', '979', '9049', '8939', '5', '9841', '522', '521', '3065', '6050', '6110', '83', '337', '5093', '6053', '7001', '8950', '321', '9070', '6233', '277', '7', '3590', '914', '32', '60', '1852', '990', '948', '8936', '1245', '62', '918', '9089', '24', '4834', '7780', '901', '6173', '3499']
['1500', '6323', '8941', '8946', '9842', '706', '9839', '5926', '382', '78', '366', '991', '2844', '922', '3655', '973', '6975', '592', '276', '7002', '1', '8944', '520', '9071', '35', '5280', '8476', '334', '9840', '148', '252', '4314', '116', '8932', '1557', '46', '981', '279', '383', '31', '483', '4', '9088', '2842', '3

  0%|          | 0/11 [00:00<?, ?it/s]

---------------------------------
Total COOPs : 11
Current COOP Name : kansas city nebraska sioux city omaha
/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/01_data/y_data/02_gc_day_level/Coop X item X week/coop=kansas city nebraska sioux city omaha.parquet


  9%|▉         | 1/11 [00:20<03:26, 20.68s/it]

---------------------------------
Total COOPs : 11
Current COOP Name : sacramento fresno reno bakersfield
/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/01_data/y_data/02_gc_day_level/Coop X item X week/coop=sacramento fresno reno bakersfield.parquet


...

---------------------------------
Total COOPs : 11
Current COOP Name : philadelphia
/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/01_data/y_data/02_gc_day_level/Coop X item X week/coop=philadelphia.parquet


 91%|█████████ | 10/11 [01:44<00:09,  9.33s/it]

---------------------------------
Total COOPs : 11
Current COOP Name : st louis
/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/01_data/y_data/02_gc_day_level/Coop X item X week/coop=st louis.parquet


100%|██████████| 11/11 [01:52<00:00, 10.23s/it]


Dataset Created and saved @  /content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/01_data/y_data/02_gc_day_level/Coop X Item X Categories/14


### 

#### Store Normalized Data

In [ ]:
# data_iter=8

# df_categories = pd.read_parquet(os.path.join(OUTPUT_DATA_ROOT,str(data_iter),'categories','train.parquet'))
# df_items = pd.read_parquet(os.path.join(OUTPUT_DATA_ROOT,str(data_iter),'items','train.parquet'))

# df_categories['str_count'] = df_categories['coop_name'].map(str_cnt_mapper)
# df_items['str_count'] = df_items['coop_name'].map(str_cnt_mapper)


In [ ]:

# categories=set(dict_cat_mapper.values())
# for col in df_categories.columns:
#   if col in categories or col=='y':
#     df_categories[col] = df_categories[col]/df_categories['str_count']
# for col in df_items.columns:
#   if 'item' in col or col=='y':
#     df_items[col] = df_items[col]/df_items['str_count']
# df_categories.drop(columns=['str_count'],inplace=True)
# df_items.drop(columns=['str_count'],inplace=True)

In [ ]:


# df_categories.to_parquet(os.path.join(OUTPUT_DATA_ROOT,str(data_iter+1),'categories','train.parquet'))
# df_items.to_parquet(os.path.join(OUTPUT_DATA_ROOT,str(data_iter+1),'items','train.parquet'))

### Scaling for category

#### Training XgBoost Model

In [ ]:


class YMTimeSeriesSplitter(object):
    @staticmethod
    def _encode(x):
        return str(x.year) + "-" + str(x.month).zfill(2)

    def _divide(self, df_train):
        train_times = [item[self._date_idx_dim] for item in
                       df_train.index.tolist()] if self._date_idx_dim is not None else df_train.index.tolist()
        train_times = pd.Series(sorted(list(set([YMTimeSeriesSplitter._encode(item) for item in train_times]))))
        self._fold_map = pd.merge(
            pd.DataFrame({'part': train_times}),
            pd.DataFrame([{
                'part': item, 'fold': fold_idx + 1
            } for fold_idx, (_, outfold_index) in enumerate(
                TimeSeriesSplit(
                    n_splits=self._nsplits,
                ).split(train_times)
            ) for item in train_times[outfold_index].tolist()
            ]),
            on='part',
            how='left'
        ).fillna(0).set_index('part').to_dict()['fold']

    def get_n_splits(self, X=None, y=None, groups=None):
        return self._nsplits

    def split(self, X=None, y=None, groups=None):
        dates = X.index.to_frame().reset_index(drop=True)
        time_col = dates.columns.tolist()[self._date_idx_dim] if self._date_idx_dim is not None else 0
        dates['fold'] = dates[time_col].apply(lambda x: self._fold_map[YMTimeSeriesSplitter._encode(x)]).astype(int)
        for fold_idx in range(1, self._nsplits + 1):
            train_index = dates[dates['fold'] < fold_idx].index.tolist()
            test_index = dates[dates['fold'] == fold_idx].index.tolist()
            yield (train_index, test_index)

    def __init__(self, train_df, date_idx_dim=0, n_splits=5) -> None:
        """
        Implements a rolling time series (year-month) based cross validation
        """
        self._nsplits = n_splits
        self._date_idx_dim = date_idx_dim
        self._divide(train_df)


class ModelTuner(object):
    def _make_grid(self, grid_filepath, kwargs):
        """
        TODO - Support for experiment_number

        Builds the grid from the grid file and the kwargs

        Grid File CSV Format:
            param_name,spec_type,spec
        """
        for rec in pd.read_csv(grid_filepath).to_dict(orient="records"):
            if rec['param_name'] in kwargs:
                continue
            if rec["spec_type"] == "range":
                self._param_grid[rec["param_name"]] = np.arange(*json.loads(rec["spec"]))
            elif rec["spec_type"] == "list":
                self._param_grid[rec["param_name"]] = json.loads(rec["spec"])
            else:
                raise ValueError("Unknown spec_type: {}".format(rec["spec_type"]))
        for key, value in kwargs.items():
            self._estimator_params[key] = value

    def _grid_search(self, train_df, actuals_col, n_splits, date_idx_dim, n_tasks, verbose):
        estimator = self._estimator_fn(**self._estimator_params)
        self._gsearch = GridSearchCV(
            estimator=estimator,
            param_grid=self._param_grid,
            scoring=self._scorer_fn,
            n_jobs=n_tasks,
            cv=YMTimeSeriesSplitter(train_df, n_splits=n_splits, date_idx_dim=date_idx_dim),
            verbose=verbose
        )
        self._gsearch.fit(train_df.drop(actuals_col, axis=1), train_df[actuals_col])

    def _build_final_model(self, df_train, actuals_col):
        final_estimator_params = {k: v for k, v in self._estimator_params.items()}
        final_estimator_params.update(self._gsearch.best_params_)
        self._best_model = self._estimator_fn(**final_estimator_params)
        self._best_model.fit(df_train.drop(actuals_col, axis=1), df_train[actuals_col])

    def get_best_model(self):
        return self._best_model

    def get_tuning_results(self):
        df = pd.DataFrame(self._gsearch.cv_results_).copy()
        for k, v in self._estimator_params.items():
            df[k] = json.dumps(v)
        return df

    def run(self, data_df, actuals_col, n_splits=3, date_idx_dim=0, n_tasks=1, verbose=3):
        self._grid_search(data_df, actuals_col, n_splits, date_idx_dim, n_tasks, verbose)
        self._build_final_model(data_df, actuals_col)
        return self

    def __init__(self, estimator_fn, grid_filepath, scorer_fn, **kwargs):
        self._estimator_fn = estimator_fn
        self._scorer_fn = scorer_fn
        self._grid_filepath = grid_filepath
        self._estimator_params = {}
        self._param_grid = {}
        self._gsearch = {}
        self._best_model = None
        self._make_grid(grid_filepath, kwargs)


class BaseModel(object):
    def y_transform_fwd(self, y):
        return y

    def y_transform_rev(self, y):
        return y

    def save(self, filepath, save_tuner=True):
        data = {
            'estimator': self._estimator,
            'control_features': self._control_features,
            'self_features': self._self_features,
            'subst_features': self._subst_features,
            'compl_features': self._compl_features,
            'no_offer_level': self._no_offer_level
        }
        if save_tuner:
            data['tuner'] = self._tuner
        with open(filepath + ".pkl" if not filepath.endswith(".pkl") else filepath, 'wb') as f:
            pickle.dump(data, f)

    def load(self, filepath):
        with open(filepath + ".pkl" if not filepath.endswith(".pkl") else filepath, 'rb') as f:
            data = pickle.load(f)
        self._estimator = data['estimator']
        self._control_features = data['control_features']
        self._self_features = data['self_features']
        self._subst_features = data['subst_features']
        self._compl_features = data['compl_features']
        self._no_offer_level = data['no_offer_level']
        if 'tuner' in data:
            self._tuner = data['tuner']
        return self

    def get_features(self, offer_only=False):
        if offer_only:
            return self._self_features + self._subst_features + self._compl_features
        return self._control_features + self._self_features + self._subst_features + self._compl_features

    def get_model(self):
        return self._estimator

    def get_tuner(self):
        return self._tuner

    def get_no_offer_level(self):
        return self._no_offer_level

    def fit(self, data_df, actuals_col, grid_filepath, date_idx_dim=0, n_tasks=1, n_threads_per_task=8, n_splits=3,
            verbose=1):
        params = {k: v for k, v in self._estimator_params.items()}
        params['n_jobs'] = n_threads_per_task
        df = data_df[self.get_features() + [actuals_col]].fillna(self._no_offer_level)
        df[actuals_col] = self.y_transform_fwd(df[actuals_col])
        self._tuner = ModelTuner(
            estimator_fn=self._estimator_fn,
            grid_filepath=grid_filepath,
            scorer_fn=self._scorer_fn,
            **params
        ).run(
            data_df=df,
            actuals_col=actuals_col,
            date_idx_dim=date_idx_dim,
            n_tasks=n_tasks,
            n_splits=n_splits,
            verbose=verbose
        )
        self._estimator = self._tuner.get_best_model()
        return self

    def predict(self, data_df, actuals_col=None, keep_other_cols=True, data_df_base=None):
        X = data_df[self.get_features()].fillna(self._no_offer_level)
        if data_df_base is not None:
            X_base = data_df_base[self.get_features()].fillna(self._no_offer_level)
        else:
            X_base = X.copy()
            X_base[self.get_features(offer_only=True)] = self._no_offer_level
        if keep_other_cols:
            result = data_df[[col for col in data_df.columns if col not in self.get_features()]].reset_index()
        else:
            result = data_df.index.to_frame()
        if actuals_col is not None and actuals_col in X:
            result['y'] = data_df[actuals_col]
        result['yhatbase'] = self._estimator.predict(X_base)
        result['yhat'] = self._estimator.predict(X)
        result['yhatbase'] = self.y_transform_rev(result['yhatbase'])
        result['yhat'] = self.y_transform_rev(result['yhat'])
        return result

    @classmethod
    def load_model(cls, filepath):
        return cls(control_features=[], self_features=[], subst_features=[], compl_features=[]).load(filepath)

    def __init__(self, control_features, self_features, subst_features, compl_features, no_offer_level=0):
        self._control_features = [col for col in control_features]
        self._self_features = [col for col in self_features]
        self._subst_features = [col for col in subst_features]
        self._compl_features = [col for col in compl_features]
        self._no_offer_level = no_offer_level
        self._tuner = None
        self._estimator = None
        self._estimator_fn = None
        self._estimator_params = {}
        self._scorer_fn = None


class GBDTRegressionModel(BaseModel):
    def _set_constraints(self):
        monotone_constraints = {}
        for feature in self._self_features:
            monotone_constraints[feature] = 1
        for feature in self._subst_features:
            monotone_constraints[feature] = -1
        for feature in self._compl_features:
            monotone_constraints[feature] = 1
        self._estimator_params['monotone_constraints'] = monotone_constraints

    def get_variable_importance(self):
        vi = pd.DataFrame({
            'feature': self.get_features(),
            'importance': self._estimator.feature_importances_
        })
        vi.loc[vi['feature'].isin(self._control_features), 'feature_type'] = 'control'
        vi.loc[vi['feature'].isin(self._self_features), 'feature_type'] = 'self'
        vi.loc[vi['feature'].isin(self._subst_features), 'feature_type'] = 'substitute'
        vi.loc[vi['feature'].isin(self._compl_features), 'feature_type'] = 'complement'
        vi['contribution'] = 100 * vi['importance'] / vi['importance'].sum()
        vi['relative_importance'] = 100 * vi['importance'] / vi['importance'].max()
        return vi[[
            'feature', 'feature_type', 'contribution', 'relative_importance'
        ]].sort_values('contribution', ascending=False).reset_index(drop=True)

    def get_split_points(self):
        tree_df = self._estimator.get_booster().trees_to_dataframe()
        tree_df = tree_df[tree_df['Feature'].isin(self.get_features(offer_only=True))][
            ['Feature', 'Split']].reset_index(drop=True)
        return tree_df.groupby('Feature')['Split'].apply(lambda x: sorted(list(set([0] + list(x))))).to_dict()

    def enable_gpu(self):
        self._estimator_params['tree_method'] = 'gpu_hist'
        self._estimator_params['predictor'] = 'gpu_predictor'

    def __init__(self, control_features, self_features, subst_features, compl_features, no_offer_level=0, **kwargs):
        super(GBDTRegressionModel, self).__init__(control_features, self_features, subst_features, compl_features,
                                                  no_offer_level)
        self._estimator_params = {
            'tree_method': 'exact',
            'random_state': 42
        }
        self._set_constraints()
        self._estimator_params.update(kwargs)
        self._estimator_fn = XGBRegressor
        self._scorer_fn = wmape_scorer



def _get_modelling_data(
        item_features_path,
        category_features_path,
        index_cols,
        coop_col,
        response_col,
        test_simulation=False
):
    train_df = pd.merge(
        pd.read_parquet(os.path.join(item_features_path, "train.parquet")).rename(columns={'coop': coop_col}),
        pd.read_parquet(os.path.join(category_features_path, "train.parquet")).rename(columns={'coop': coop_col}),
        on=index_cols + [coop_col, response_col]
    ).fillna(0)

    if test_simulation:

      test_df = pd.merge(
          pd.read_parquet(os.path.join(item_features_path, "test.parquet")).rename(columns={'coop': coop_col}),
          pd.read_parquet(os.path.join(category_features_path, "test.parquet")).rename(columns={'coop': coop_col}),
          on=index_cols + [coop_col]
      ).fillna(0)

    else:
      test_df = pd.DataFrame()

    return train_df, test_df


def fit_model(gc_item_features_path, gc_category_features_path, index_cols, coop_col, response_col, grid_filepath,
              xgb_num_tasks, xgb_num_threads_per_task, iteration_root):
    train_df, _ = _get_modelling_data(
        item_features_path=gc_item_features_path,
        category_features_path=gc_category_features_path,
        index_cols=index_cols,
        coop_col=coop_col,
        response_col=response_col
    )


    self_features = [col for col in train_df.columns if
                     col in dict_feature_mapper.values() or col in dict_cat_mapper.values()]
    model = GBDTRegressionModel(
        self_features=self_features,
        control_features=[],
        subst_features=[],
        compl_features=[],
        no_offer_level=0
    )
    if 'google.colab' in sys.modules:
        model.enable_gpu()
    model = model.fit(
        data_df=train_df.set_index(index_cols),
        actuals_col=response_col,
        grid_filepath=grid_filepath,
        n_tasks=xgb_num_tasks,
        n_threads_per_task=xgb_num_threads_per_task
    )
    model.save(os.path.join(iteration_root, "model.pkl"))
    model.get_tuner().get_tuning_results().to_csv(
        os.path.join(iteration_root, "tuning_results.csv")
    )
    model.get_variable_importance().to_csv(os.path.join(iteration_root, "vi.csv"))
    model.get_model().save_model(os.path.join(iteration_root, "model.ubj"))
    train_predictions = model.predict(train_df.set_index(index_cols))


    train_df['yhat'] = train_predictions['yhat']


    train_predictions['split'] = 'train'
    train_predictions['y'] = train_df['y']
    
    #test_predictions = model.predict(test_df.set_index(index_cols))

    #test_df['y']=0
    #test_df['yhat'] = test_predictions['yhat']

    #test_predictions['split'] = 'test'

    
    #test_predictions['y'] = 0


    #predictions = pd.concat([train_predictions, test_predictions])

    # print(train_predictions)
    # print(test_predictions)
    # print(predictions)

    # metrics = pd.merge(
    #     train_df.groupby(coop_col).apply(lambda x: wmape(x['y'], x['yhat'])).reset_index().rename(
    #         columns={0: 'wmape_train'}),
    #     test_df.groupby(coop_col).apply(lambda x: wmape(x['y'], x['yhat'])).reset_index().rename(
    #         columns={0: 'wmape_test'}),
    #     on=coop_col
    # )

    metrics = train_df.groupby(coop_col).apply(lambda x: wmape(x['y'], x['yhat'])).reset_index().rename(
           columns={0: 'wmape_train'})
    
    train_predictions.reset_index().to_csv(os.path.join(iteration_root, "predictions.csv"))
    metrics.to_csv(os.path.join(iteration_root, "metrics.csv"))
    return model
    
def multiindex_pivot(df, index=None, columns=None, values=None):
    # https://github.com/pandas-dev/pandas/issues/23955
    if index is None:
        names = list(df.index.names)
        df = df.reset_index()
    else:
        names = index
    list_index = df[names].values
    tuples_index = [tuple(i) for i in list_index]  # hashable
    df = df.assign(tuples_index=tuples_index)
    df = df.pivot(index="tuples_index", columns=columns, values=values)
    tuples_index = df.index  # reduced
    index = pd.MultiIndex.from_tuples(tuples_index, names=names)
    df.index = index
    return df


def _predict_from_demand(
    model,
    demand_predictions_path,
    date_col,
    coop_col,
    cat_mapper,
    index_cols,
    baseline,
    corrected_base,
    store_level
):
    if baseline:
        if corrected_base:
            pred_col = "yhatbase_corrected"
        else:
            pred_col = "yhatbase"
    else:
        pred_col = "yhat"

    gc_input = pd.read_parquet(demand_predictions_path)
    
    
    
    gc_input["feature_name"] = gc_input["dv_id"].apply(lambda x: "item_" + str(x))
    gc_input = gc_input[
        [date_col, coop_col, "feature_name", pred_col, "split"]


    ].reset_index(drop=True)
    gc_input["estimated_measure"] = "gc"
    gc_input["category"] = gc_input["feature_name"].map(cat_mapper)

    # base_gc_items=base_gc.pipe(multiindex_pivot, index=index_cols + [coop_col], columns='feature_name', values='yhatbase').fillna(0).reset_index()
    # gc_input_items = gc_input.pivot(
    #     index=index_cols + [coop_col,'split'],
    #     columns="feature_name",
    #     values=pred_col
    # ).fillna(0).reset_index()

    gc_input_items = (
        gc_input.pipe(
            multiindex_pivot,
            index=index_cols + [coop_col, "split"],
            columns="feature_name",
            values=pred_col,
        )
        .fillna(0)
        .reset_index()
    )


    # gc_input_cat = gc_input[index_cols + [coop_col, 'category', pred_col,'split']].groupby(
    #     index_cols + [coop_col, 'category','split']
    # )[pred_col].sum().reset_index().pivot(
    #     index=index_cols + [coop_col,'split'],
    #     columns='category',
    #     values=pred_col
    # ).fillna(0).reset_index()
    
    ## rescaling for categories 
    #gc_input = pd.read_parquet(demand_predictions_path)
    #gc_input= scaling_for_category(gc_input,ITEM_MASTER,pred_col)
    #gc_input["feature_name"] = gc_input["dv_id"].apply(lambda x: "item_" + str(x))
    # gc_input = gc_input[
    #     [date_col, coop_col, "feature_name", pred_col, "split"] ]

    # gc_input["estimated_measure"] = "gc"
    gc_input["category"] = gc_input["feature_name"].map(cat_mapper)

    gc_input_cat = (
        gc_input[index_cols + [coop_col, "category", pred_col, "split"]]
        .groupby(index_cols + [coop_col, "category", "split"])[pred_col]
        .sum()
        .reset_index()
        .pipe(
            multiindex_pivot,
            index=index_cols + [coop_col, "split"],
            columns="category",
            values=pred_col,
        )
        .fillna(0)
        .reset_index()
    )

    if store_level:
      gc_input_items['str_count']= gc_input_items[coop_col].map(str_cnt_mapper)
      gc_input_cat['str_count']= gc_input_cat[coop_col].map(str_cnt_mapper)
      
      for col in gc_input_items.columns:
        if 'item' in col:
          gc_input_items[col] = gc_input_items[col]/gc_input_items['str_count']

      
      for col in gc_input_cat.columns:
        if col in set(cat_mapper.values()):
          gc_input_cat[col] =  gc_input_cat[col]/gc_input_cat['str_count']

      gc_input_items.drop(columns=['str_count'],inplace=True)
      gc_input_cat.drop(columns=['str_count'],inplace=True)
      
      


    gc_input = pd.merge(gc_input_items, gc_input_cat, on=index_cols + [coop_col, "split"])
    gc_input = pd.concat(
        [
            gc_input,
            pd.DataFrame(
                {
                    col: [0.0] * gc_input.shape[0]
                    for col in model.get_features()
                    if col not in gc_input
                }
            ),
        ],
        axis=1,
    )

    gc_input[pred_col] = model.get_model().predict(gc_input[model.get_features()])

    return gc_input[index_cols + [coop_col, pred_col, "split"]]


def get_gc_predictions(
    model,
    demand_predictions_path,
    date_col,
    coop_col,
    cat_mapper,
    index_cols,
    store_level,
    corrected_base=True
    
):

    result = pd.merge(
        _predict_from_demand(
            model,
            demand_predictions_path,
            date_col,
            coop_col,
            cat_mapper,
            index_cols,
            baseline=True,
            corrected_base=corrected_base,
            store_level=store_level
        ),
        _predict_from_demand(
            model,
            demand_predictions_path,
            date_col,
            coop_col,
            cat_mapper,
            index_cols,
            baseline=False,
            corrected_base=corrected_base,
            store_level=store_level
        ),
        on=index_cols + [coop_col, "split"],
    )

    result = result.sort_values(by=["coop_name", "pos_busn_dt"], ascending=True)

    result["pos_busn_dt"] = pd.to_datetime(result["pos_busn_dt"])
    return result


def _get_base_gc(
    model,
    demand_predictions_path,
    date_col,
    coop_col,
    cat_mapper,
    index_cols,
    store_level,
    corrected_base=True
    
):

    base_gc = pd.read_parquet(demand_predictions_path).query('split=="train"')

    if corrected_base:
        base_gc = base_gc.drop(columns=["yhatbase"]).rename(
            columns={"yhatbase_corrected": "yhatbase"}
        )
    else:
        pass

    base_gc["feature_name"] = base_gc["dv_id"].apply(lambda x: "item_" + str(x))
    base_gc = base_gc[[date_col, coop_col, "feature_name", "yhatbase"]].reset_index(
        drop=True
    )
    base_gc["estimated_measure"] = "gc"
    


    base_gc_items = (
        base_gc.pipe(
            multiindex_pivot,
            index=index_cols + [coop_col],
            columns="feature_name",
            values="yhatbase",
        )
        .fillna(0)
        .reset_index()
    )

    # base_gc_items = base_gc.pivot(
    #     index=index_cols + [coop_col],
    #     columns="feature_name",
    #     values="yhatbase"
    # ).fillna(0).reset_index()

    # base_gc_cat = base_gc[index_cols + [coop_col, 'category', 'yhatbase']].groupby(
    #     index_cols + [coop_col, 'category'])['yhatbase'].sum().reset_index().pivot(
    #     index=index_cols + [coop_col],
    #     columns='category',
    #     values='yhatbase'
    # ).fillna(0).reset_index()

    # base_gc = pd.read_parquet(demand_predictions_path).query('split=="train"')
    # base_gc=scaling_for_category(base_gc,ITEM_MASTER,'yhatbase_corrected')

    # if corrected_base:
    #     base_gc = base_gc.drop(columns=["yhatbase"]).rename(
    #         columns={"yhatbase_corrected": "yhatbase"}
    #     )
    # else:
    #     pass

    # base_gc["feature_name"] = base_gc["dv_id"].apply(lambda x: "item_" + str(x))  
    # base_gc = base_gc[[date_col, coop_col, "feature_name", "yhatbase"]].reset_index(
    #     drop=True
    # )
    # base_gc["estimated_measure"] = "gc"
      
    base_gc["category"] = base_gc["feature_name"].map(cat_mapper)

    base_gc_cat = (
        base_gc[index_cols + [coop_col, "category", "yhatbase"]]
        .groupby(index_cols + [coop_col, "category"])["yhatbase"]
        .sum()
        .reset_index()
        .pipe(
            multiindex_pivot,
            index=index_cols + [coop_col],
            columns="category",
            values="yhatbase",
        )
        .fillna(0)
        .reset_index()
    )

    base_gc = pd.merge(base_gc_items, base_gc_cat, on=index_cols + [coop_col])


    base_gc = pd.concat(
        [
            base_gc,
            pd.DataFrame(
                {
                    col: [0.0] * base_gc.shape[0]
                    for col in model.get_features()
                    if col not in base_gc
                }
            ),
        ],
        axis=1,
    )
    

    if store_level:
      base_gc['str_cnt'] =base_gc['coop_name'].map(str_cnt_mapper)
      for col in base_gc.columns:
        if col not in ['coop_name','estimated_measure','pos_busn_dt']:

          base_gc[col]= base_gc[col]/base_gc['str_cnt']
      
      base_gc.drop(columns=['str_cnt'],inplace=True)





    base_gc["yhatbase"] = model.get_model().predict(base_gc[model.get_features()])
    return base_gc



def simulate_gc_from_predictions(
    model,
    demand_predictions_path,
    demand_tensor_path,
    date_col,
    coop_col,
    index_cols,
    cat_mapper,
    corrected_base,
    store_level,
    cold_start=None,
    coop=None
   
):
    result = []
    logger = set()

    # df_demand_tensor = pd.read_csv(demand_tensor_path).query(
    #     'dv_id != -1 and not (source=="cross-learning" and effect=="cross")').reset_index(drop=True).rename(columns={'coop':coop_col}).reset_index(drop=True)
    
    df_demand_tensor = (
        pd.read_csv(demand_tensor_path).query("dv_id != -1").reset_index(drop=True)
    )

    if coop is not None:
        df_demand_tensor = df_demand_tensor.query("coop_name==@coop").reset_index(
            drop=True
        )

    df_demand_tensor["feature_name"] = df_demand_tensor["dv_id"].apply(
        lambda x: "item_" + str(x)
    )
   

    ##get average for november and march 
    base_gc = _get_base_gc(
        model=model,
        demand_predictions_path=demand_predictions_path,
        date_col=date_col,
        coop_col=coop_col,
        cat_mapper=cat_mapper,
        index_cols=index_cols,
        corrected_base=corrected_base,
        store_level=store_level
    )
    
    for (coop_name, idv_id, price_point, per_depth), sub_tensor in tqdm(
        df_demand_tensor.groupby(["coop_name", "idv_id", "price_point", "depth_per"])
    ):
        scenario_gc = base_gc.copy()

        for rec in sub_tensor.to_dict(orient="records"):
            if rec["feature_name"] in base_gc.columns:

                delta = base_gc[rec["feature_name"]] * (
                    rec["elasticity"] * rec["depth_per"]
                )
                scenario_gc[rec["feature_name"]] += delta
                scenario_gc[cat_mapper[rec["feature_name"]]] += delta
            else:
                logger.add(rec["feature_name"])

                continue

        scenario_gc["yhat"] = model.get_model().predict(scenario_gc[model.get_features()])
        fdf = scenario_gc[scenario_gc[coop_col] == coop_name]
        
        yhat = fdf["yhat"].mean()
        yhatbase = fdf["yhatbase"].mean()
        per_gc = (yhat - yhatbase) / yhatbase
        elasticity = per_gc / per_depth

        result.append(
            {
                "yhat": yhat,
                "yhatbase": yhatbase,
                "coop_name": coop_name,
                "idv_id": idv_id,
                "price_point": price_point,
                "depth_per": per_depth,
                "impact": per_gc,
                "elasticity": elasticity,
            }
        )
    result = pd.DataFrame(result)
    result = pd.merge(
        df_demand_tensor[
            [
                "coop_name",
                "idv_id",
                "daypart",
                "price_point",
                # 'matching_level',
                # 'offer_level_base_price',
                "offer_price",
                "dis_mech",
                "item_grp",
            ]
        ]
        .drop_duplicates()
        .reset_index(drop=True),
        result,
        on=["coop_name", "idv_id", "price_point"],
        how="left",
    )
    result["dv_id"] = -1
    result = (
        result.sort_values(["coop_name", "idv_id", "price_point"])
        .reset_index(drop=True)
        .fillna(0)
        .drop_duplicates()
    )
    if cold_start is not None:
        result = pd.concat(
            [
                pd.read_csv(cold_start)
                .query("coop_name != @coop")
                .reset_index(drop=True),
                result,
            ]
        )
    return result, logger


def _simulate_per(model, train_df, yhatbase, feature, category, percentages, coop_col):
    result = []
    for xper in percentages:
        try:
            X = train_df[model.get_features()].copy()
            delta_counts = xper * X[feature]
            X[feature] += delta_counts
            X[category] += delta_counts
            yper = ((model.get_model().predict(X) - yhatbase) / yhatbase).tolist()
            rdf = pd.DataFrame({
                coop_col: train_df[coop_col].tolist(),
                'per_gc': yper,
            })
            rdf['feature_name'] = feature
            rdf['per_qty'] = xper
            gcols = ["feature_name", coop_col, "per_qty"]
            mdf = pd.merge(
                rdf.groupby(gcols)["per_gc"].mean().reset_index(),
                rdf.groupby(gcols)["per_gc"].count().reset_index().rename(columns={'per_gc': 'count'}),
                on=gcols
            )
            result.append(mdf)
        except:
            # TODO Get rid of this by fixing missing stuff
            pass
    return result


def simulate_gc_from_actuals(model, gc_item_features_path, gc_category_features_path, demand_tensor_path, index_cols,
                             coop_col, response_col, cat_mapper):
    train_df, _ = _get_modelling_data(
        item_features_path=gc_item_features_path,
        category_features_path=gc_category_features_path,
        index_cols=index_cols,
        coop_col=coop_col,
        response_col=response_col
    )
    df_demand_tensor = pd.read_csv(demand_tensor_path).query('dv_id != -1').reset_index(drop=True)
    df_demand_tensor['feature_name'] = df_demand_tensor['dv_id'].apply(lambda x: "item_" + str(x))
    #percentages = df_demand_tensor.groupby(['feature_name'])['impact'].apply(lambda x: list(set(x))).to_dict()

    print(percentages)
    yhatbase = model.get_model().predict(train_df[model.get_features()])
    result = []
    for feature in tqdm(model.get_features()):
        if feature in cat_mapper and feature in percentages:
            category = cat_mapper[feature]
            result += _simulate_per(model, train_df, yhatbase, feature, category, percentages[feature], coop_col)
    result = pd.concat(result, ignore_index=True)
    result['elasticity'] = result['per_gc'] / result['per_qty']
    result = pd.merge(
        df_demand_tensor.rename(columns={'caliberated_elas': 'demand_elasticity','coop':'coop_name'}),
        result.drop(columns=['per_gc', 'count']).rename(
            columns={'per_qty': 'impact', 'elasticity': 'gc_elasticity_item'}),
        on=['feature_name', 'coop_name', 'impact'],
        how='left'
    )
    result['elasticity'] = result['gc_elasticity_item'] * result['demand_elasticity']
    result = result.groupby([
        'coop_name',
        'idv_id',
        'daypart',
        'price_point',
        'base_price',
        'dis_mech',
        'item_grp',
        'depth_per',
    ])['elasticity'].sum().reset_index()
    result['impact'] = result['elasticity'] * result['depth_per']
    return result.sort_values(['coop_name', 'idv_id', 'price_point']).reset_index(drop=True)


#### Training/Loading Pretrained Model

In [ ]:
cfg_refit_model

False

In [ ]:

MODEL_ITERATION_ROOT

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/03_gc_models/xgb_models/18'

In [ ]:
fp_gc_item_features

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/01_data/y_data/02_gc_day_level/Coop X Item X Categories/14/items'

In [ ]:
fp_gc_category_features


'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/01_data/y_data/02_gc_day_level/Coop X Item X Categories/14/categories'

In [ ]:
%%time


if cfg_refit_model:
  xgb_model = fit_model(
          gc_item_features_path=fp_gc_item_features,
          gc_category_features_path=fp_gc_category_features,
          index_cols=cl_index_cols,
          coop_col=cl_coop_col,
          response_col=cl_response_col,
          grid_filepath=fp_grid,
          xgb_num_tasks=cfg_xgb_num_tasks,
          xgb_num_threads_per_task=cfg_xgb_num_threads_per_task,
          iteration_root=MODEL_ITERATION_ROOT
      )
else:
    xgb_model = GBDTRegressionModel(
        self_features=[],
        control_features=[],
        subst_features=[],
        compl_features=[],
        no_offer_level=0
    ).load_model(
        os.path.join(MODEL_ITERATION_ROOT, "model.pkl")
    )

CPU times: user 13.2 ms, sys: 529 ms, total: 542 ms
Wall time: 1.56 s


In [ ]:
xgb_model = GBDTRegressionModel(
        self_features=[],
        control_features=[],
        subst_features=[],
        compl_features=[],
        no_offer_level=0
    )
xgb_model._estimator = xgb.Booster()
xgb_model._estimator.load_model(path_to_ubj)
xgb_model._self_features = xgb_model._feature_names

In [ ]:
fp_gc_item_features

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/01_data/y_data/02_gc_day_level/Coop X Item X Categories/14/items'

In [ ]:
fp_gc_category_features


'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/01_data/y_data/02_gc_day_level/Coop X Item X Categories/14/categories'

In [ ]:

MODEL_ITERATION_ROOT

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/03_gc_models/xgb_models/18'

In [ ]:
fp_demand_predictions

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/41/01_demand/model_collated/model_predictions.parquet'

In [ ]:
output_tensor_path

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/41/04_tensor/tensor_processed/all_tensor_processed_raw.csv'

### Generate GC Elasticity and Predictions 

In [ ]:
result, lgr = simulate_gc_from_predictions(
      model=xgb_model,
      demand_predictions_path=fp_demand_predictions,
      demand_tensor_path=output_tensor_path,
      date_col="pos_busn_dt",
      coop_col="coop_name",
      index_cols=["pos_busn_dt", "estimated_measure"],
      cat_mapper=dict_cat_mapper,
      corrected_base=True,
      store_level=False
  )

100%|██████████| 8590/8590 [05:31<00:00, 25.88it/s]


In [ ]:
output_gc_simulation_filepath

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/41/06_gc_xgb/gc_simulation_method_1.csv'

In [ ]:
# output_gc_simulation_filepath.replace('gc_simulation_method_1.csv','14/gc_simulation_method_1.csv')

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/31/06_gc_xgb/14/gc_simulation_method_1.csv'

In [ ]:
# output_gc_simulation_filepath

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/33/06_gc_xgb/gc_simulation_method_1.csv'

In [ ]:
result.to_csv(output_gc_simulation_filepath,index=False)

In [ ]:
store_level=True

In [ ]:
output_gc_simulation_filepath

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/41/06_gc_xgb/gc_simulation_method_1.csv'

In [ ]:
exp_7 ='/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/25/06_gc_xgb/exp-7/gc_simulation_method_1.csv'

In [ ]:
exp_8 ='/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/25/06_gc_xgb/exp-8/gc_simulation_method_1.csv'

In [ ]:

#result.to_csv(exp_8, index=None)


In [ ]:
fp_demand_predictions

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/33/01_demand/model_collated/model_predictions.parquet'

In [ ]:
MODEL_ITERATION_ROOT


'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/03_gc_models/xgb_models/16'

In [ ]:
  

# if save_gc_elasticitity:
#   result.to_csv(output_gc_predictions_filepath, index=None)

gc_predictions = get_gc_predictions(
      model=xgb_model,
      demand_predictions_path=fp_demand_predictions,
      date_col="pos_busn_dt",
      coop_col="coop_name",
      cat_mapper=dict_cat_mapper,
      index_cols=["pos_busn_dt", "estimated_measure"],
      corrected_base=True,
      store_level=False)

# if save_gc_predictions:
#   gc_predictions.to_csv(output_gc_predictions_filepath, index=None)

In [ ]:
# output_gc_predictions_filepath.replace('model_predictions.csv','14/model_predictions.csv')

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/31/06_gc_xgb/14/model_predictions.csv'

In [ ]:
output_gc_predictions_filepath

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/41/06_gc_xgb/model_predictions.csv'

In [ ]:
gc_predictions.to_csv(output_gc_predictions_filepath,index=False)

#### Create GC DEBUG Notebook

In [ ]:
actual_gc =pd.read_parquet(GC_DATA_PATH_WEEKLY.replace('coop=','')).sort_values(['coop','pos_busn_dt']).rename(columns={'total_gc':'actual_gc','coop':'coop_name'})
actual_gc['pos_busn_dt']=pd.to_datetime(actual_gc['pos_busn_dt'])
# gc_predicted_merge = gc_predictions.merge(actual_gc,on=['coop_name','pos_busn_dt'],how='left')
# gc_predicted_merge['store_count'] =gc_predicted_merge['coop_name'].map(str_cnt_mapper)
# gc_predicted_merge['yhat_per_store'] = gc_predicted_merge['yhat']/ gc_predicted_merge['store_count']
# gc_predicted_merge['yhatbase_per_store'] =  gc_predicted_merge['yhatbase']/ gc_predicted_merge['store_count']
# gc_predicted_merge['actual_gc_per_store'] = gc_predicted_merge['actual_gc']/ gc_predicted_merge['store_count']

In [ ]:
gc_predictions['str_count'] =gc_predictions['coop_name'].map(str_cnt_mapper)

In [ ]:
gc_predictions = gc_predictions.merge(actual_gc,on=['coop_name','pos_busn_dt'],how='left')

In [ ]:
output_gc_predictions_filepath

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/26/06_gc_xgb/model_predictions.csv'

In [ ]:
gc_predictions.to_csv("/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/26/06_gc_xgb/model_predictions_decorr.csv",index=False)

In [ ]:
#gc_predictions.to_csv(exp_7.replace('gc_simulation_method_1.csv','model_predictions_7.csv'),index=False)

In [ ]:
gc_predictions['yhatbase_corrected_coop'] = gc_predictions['yhatbase_corrected']*gc_predictions['str_count']
gc_predictions['yhat_coop']= gc_predictions['yhat']*gc_predictions['str_count']


In [ ]:
gc_predictions.to_csv(output_gc_predictions_filepath.replace('/model_predictions.csv','/per_store/model_predictions.csv'),index=False)

In [ ]:
gc_predicted_merge = gc_predictions.merge(actual_gc,on=['coop_name','pos_busn_dt'],how='left')

In [ ]:
gc_predicted_merge

,pos_busn_dt,estimated_measure,coop_name,yhatbase_corrected,split,yhat,str_count,yhatbase_corrected_coop,yhat_coop,actual_gc
0,2019-01-07,gc,albuquerque el paso,7237.396484,train,7348.896484,132.0,9.553363e+05,9.700543e+05,961324.0
1,2019-01-14,gc,albuquerque el paso,7148.249512,train,7230.482910,132.0,9.435689e+05,9.544237e+05,962712.0
2,2019-01-21,gc,albuquerque el paso,7085.852539,train,7162.291992,132.0,9.353325e+05,9.454225e+05,952774.0
3,2019-01-28,gc,albuquerque el paso,7234.362793,train,7238.036133,132.0,9.549359e+05,9.554208e+05,988217.0
4,2019-02-04,gc,albuquerque el paso,7520.579102,train,7561.579590,132.0,9.927164e+05,9.981285e+05,996475.0
...,...,...,...,...,...,...,...,...,...,...
2404,2023-07-03,gc,washington dc baltimore eastern shore,8177.697266,test,8177.697266,460.0,3.761741e+06,3.761741e+06,NaN
2405,2023-07-10,gc,washington dc baltimore eastern shore,8171.047363,test,8171.047363,460.0,3.758682e+06,3.758682e+06,NaN
2406,2023-07-17,gc,washington dc baltimore eastern shore,8159.281738,test,8159.281738,460.0,3.753270e+06,3.753270e+06,NaN
2407,2023-07-24,gc,washington dc baltimore eastern shore,8236.229492,test,8236.229492,460.0,3.788666e+06,3.788666e+06,NaN


In [ ]:
gc_predicted_merge.to_csv(output_gc_predictions_filepath.replace('/model_predictions.csv','/per_store/model_predictions_debug.csv'),index=False)

In [ ]:
def get_mapes(df,y1,y2):
  metrics = df.groupby('coop_name').apply(lambda x: wmape(x[y1], x[y2])).reset_index().rename(
           columns={0: 'wmape_train'})
  return metrics    

### comparison

In [ ]:
#old_predictions =pd.read_csv(output_gc_predictions_filepath)
#old_predictions['pos_busn_dt']=pd.to_datetime(old_predictions['pos_busn_dt'])

In [ ]:
old_predictions = old_predictions.merge(actual_gc,on=['coop_name','pos_busn_dt'],how='left')

In [ ]:
metrics = get_mapes(old_predictions,'yhatbase_corrected','actual_gc')

In [ ]:
#metrics.to_csv(output_gc_predictions_filepath.replace('model_predictions.csv','model_metrics.csv'),index=False)

In [ ]:
new_predictions = pd.read_csv(output_gc_predictions_filepath.replace('/model_predictions.csv','/per_store/model_predictions_debug.csv'))
new_predictions['pos_busn_dt']=pd.to_datetime(new_predictions['pos_busn_dt'])

In [ ]:
new_predictions

,pos_busn_dt,estimated_measure,coop_name,yhatbase_corrected,split,yhat,str_count,yhatbase_corrected_coop,yhat_coop,actual_gc
0,2019-01-07,gc,albuquerque el paso,7237.3965,train,7348.8965,132.0,9.553363e+05,9.700543e+05,961324.0
1,2019-01-14,gc,albuquerque el paso,7148.2495,train,7230.4830,132.0,9.435689e+05,9.544237e+05,962712.0
2,2019-01-21,gc,albuquerque el paso,7085.8525,train,7162.2920,132.0,9.353325e+05,9.454225e+05,952774.0
3,2019-01-28,gc,albuquerque el paso,7234.3630,train,7238.0360,132.0,9.549359e+05,9.554208e+05,988217.0
4,2019-02-04,gc,albuquerque el paso,7520.5790,train,7561.5796,132.0,9.927164e+05,9.981285e+05,996475.0
...,...,...,...,...,...,...,...,...,...,...
2404,2023-07-03,gc,washington dc baltimore eastern shore,8177.6973,test,8177.6973,460.0,3.761741e+06,3.761741e+06,NaN
2405,2023-07-10,gc,washington dc baltimore eastern shore,8171.0474,test,8171.0474,460.0,3.758682e+06,3.758682e+06,NaN
2406,2023-07-17,gc,washington dc baltimore eastern shore,8159.2817,test,8159.2817,460.0,3.753270e+06,3.753270e+06,NaN
2407,2023-07-24,gc,washington dc baltimore eastern shore,8236.2295,test,8236.2295,460.0,3.788666e+06,3.788666e+06,NaN


In [ ]:
metrics_new = get_mapes(new_predictions,'yhat_coop','actual_gc')

In [ ]:
#metrics_new.to_csv(output_gc_predictions_filepath.replace('/model_predictions.csv','/per_store/model_metrics.csv'),index=False)

#### Get Normalized Categories

In [ ]:
fp_demand_predictions

'/content/drive/Shareddrives/MCD_Discount Engine/US/2022/4. Modeling/99_pipeline/31/01_demand/model_collated/model_predictions.parquet'

In [ ]:
df_demand_predictions.query('coop_name=="denver col springs s colorado"')

,estimated_measure,dv_id,pos_busn_dt,y,yhat,yhatbase,algo,split,coop_name,yhatbase_corrected,item_category
0,item_qty,1,2019-01-07,222.151220,259.498349,279.066332,lasso,train,denver col springs s colorado,279.066332,ENTREES
1,item_qty,1,2019-01-14,228.931707,259.109399,278.677381,lasso,train,denver col springs s colorado,278.677381,ENTREES
2,item_qty,1,2019-01-21,220.960976,258.960507,278.528489,lasso,train,denver col springs s colorado,278.528489,ENTREES
3,item_qty,1,2019-01-28,247.639024,278.078261,278.078261,lasso,train,denver col springs s colorado,278.078261,ENTREES
4,item_qty,1,2019-02-04,235.190244,274.750221,274.750221,lasso,train,denver col springs s colorado,274.750221,ENTREES
...,...,...,...,...,...,...,...,...,...,...,...
35467,None,9842,2024-01-29,0.000000,138.599736,138.599736,lasso,test,denver col springs s colorado,138.599736,BREAKFAST MEALS
35468,None,9842,2024-02-05,0.000000,140.877880,140.877880,lasso,test,denver col springs s colorado,140.877880,BREAKFAST MEALS
35469,None,9842,2024-02-12,0.000000,140.600294,140.600294,lasso,test,denver col springs s colorado,140.600294,BREAKFAST MEALS
35470,None,9842,2024-02-19,0.000000,141.438925,141.438925,lasso,test,denver col springs s colorado,141.438925,BREAKFAST MEALS


In [ ]:
import pandas as pd
A = ['Apple', 'Dog']
B = ['Cat', 'OWL', 'PEACOCK']
df1 = pd.DataFrame({'A':A})
df2 = pd.DataFrame({'B':B})
pd.concat([df1,df2],axis=1).to_csv('myfile.csv', index = False)